In [1]:
from __future__ import print_function

Load in the Python modules necessary to load the data (gzip and yaml), process the data (numpy and pandas), and plot the data (toyplot).

In [2]:
import gzip
import yaml
import numpy
import pandas
import toyplot.pdf

print("yaml version:    ", yaml.__version__)
print("numpy version:   ", numpy.__version__)
print("pandas version:  ", pandas.__version__)
print("toyplot version: ", toyplot.__version__)

yaml version:     3.12
numpy version:    1.13.3
pandas version:   0.20.3
toyplot version:  0.16.0


## Ingest Data

Load the data, which is output into a YAML file. (Actually, there are several runs that have been concatinated into a YAML file that has a sequence in its top level.)

In [3]:
filename = 'miniGraphics-skybridge-vn-no-compress.yaml.gz'
yaml_data = yaml.load(gzip.open(filename))
data = pandas.DataFrame(yaml_data)

The YAML data is hierarchical. The basic yaml reader to DataFrame just embeds dictionaries and lists in DataFrame columns. Fix that by expanding the data of these columns into new columns.

In [4]:
def expand_single_column(original_data, column_to_expand):

    expanded_data = pandas.DataFrame()
    for index in original_data.index:
        sub_table = pandas.DataFrame(original_data[column_to_expand][index])
        for column in original_data.columns:
            if column != column_to_expand:
                sub_table[column] = numpy.full(sub_table.index.shape,
                                               original_data[column][index],
                                               dtype=original_data[column].dtype)
        expanded_data = expanded_data.append(sub_table, ignore_index=True)
    return expanded_data

def flatten_table(original_data):
    flat_data = original_data
    for column_name in original_data.columns:
        if isinstance(flat_data[column_name][0], list):
            flat_data = expand_single_column(flat_data, column_name)
    return flat_data

In [5]:
data = flatten_table(data)

Add a column that gives a human-readable name to each image resolution.

In [6]:
image_height_names = {
    500: 'Desktop Window',
    1080: 'HDTV',
    4320: '8K UHD',
}

data['image-size'] = data['image-height'].map(image_height_names)

Rename the algorithms from the identifiers the program wrote out to the strings used in the paper. Note that there are some extras in the data that we are ignoring.

In [7]:
algorithm_names = {
    '2-3-SwapBase': '2-3 Swap',
    'BinarySwapFold': 'Naive',
    'BinarySwapTelescoping': 'Telescoping',
    'BinarySwapRemainder': 'Remainder',
    'IceTBase': 'IceT'
}

data['composite-algorithm'] = data['composite-algorithm'].map(algorithm_names)

Print a summary of the table data. There are multiple ways that Jupyter and pandas will report a summary of a table, but I find this method the most effective. It prints out every column. Then for all columns with a "small" number of unique values, it gives those values. This latter information really helps identify the proper way to group values.

In [8]:
import IPython.display

data_description = ''

for column_name in data.columns:
    data_description = data_description + '**' + column_name + '**: '
    unique_values = data[column_name].unique()
    if (len(unique_values) < 10):
        for value in unique_values:
            data_description = data_description + str(value) + ' '
    elif (numpy.issubdtype(unique_values.dtype, numpy.number)):
        data_description = (
            data_description +
            str(numpy.nanmin(unique_values)) + ' &ndash; ' +
            str(numpy.nanmax(unique_values)) + ' '
        )
    elif not pandas.isnull(unique_values).any():
        data_description = (
            data_description +
            str(numpy.min(unique_values)) + ' &ndash; ' +
            str(numpy.max(unique_values)) + ' '
        )
    data_description = data_description + ' \n'
    
IPython.display.display(IPython.display.Markdown(data_description))

**color-buffer-format**: byte  
**composite-algorithm**: Naive Remainder Telescoping 2-3 Swap IceT  
**composite-seconds**: 0.00118646 &ndash; 4.17276  
**construct-tree-seconds**: 0.000183397 &ndash; 0.0731399  
**depth-buffer-format**: float  
**gather-seconds**: 0.00068552 &ndash; 2.67005  
**geometry**: box  
**geometry-distribution**: duplicate  
**geometry-overlap**: -0.05  
**icet-copy-result-seconds**: 1.052e-06 &ndash; 0.00239137  
**image-compression**: False  
**image-height**: 288 1080 4320  
**image-width**: 512 1920 7680  
**num-processes**: 128 &ndash; 8192  
**num-triangles**: 1536 &ndash; 98304  
**paint-seconds**: 0.000118637 &ndash; 0.141582  
**painter**: simple  
**partial-composite-seconds**: 0.000291541 &ndash; 2.07523  
**phi-rotation**: -178.406 &ndash; 136.756  
**random-seed**: 17627  
**rendering-order-dependent**: False  
**start-time**: 2018-05-24T17:07:04.000000000 &ndash; 2018-05-29T07:30:55.000000000  
**theta-rotation**: -177.945 &ndash; 179.418  
**total-seconds**: 0.00144766 &ndash; 4.31646  
**trial-num**: 0 &ndash; 19  
**zoom**: 1.5  
**image-size**: nan HDTV 8K UHD  


## Plot Data

We are plotting the time it takes to do a "partial composite" (that is the time to blend all the pixels, but the pixels are left distributed across all the processes).

The first thing we want to do is to average the time it took over all trials. This is easily done with a pivot table.

In [9]:
average_partial_composite = data.pivot_table(
    values='partial-composite-seconds',
    index='num-processes',
    columns=[
        'image-size',
        'composite-algorithm',
    ],
    aggfunc='mean',
)

print(average_partial_composite.index)
print(average_partial_composite.columns)

Int64Index([ 128,  160,  176,  192,  224,  256,  272,  320,  352,  400,  448,
             512,  560,  640,  720,  800,  912, 1024, 1136, 1280, 1440, 1616,
            1824, 2048, 2288, 2576, 2896, 3248, 3648, 4096, 4592, 5152, 5792,
            6496, 7296, 8192],
           dtype='int64', name=u'num-processes')
MultiIndex(levels=[[u'8K UHD', u'HDTV'], [u'2-3 Swap', u'IceT', u'Naive', u'Remainder', u'Telescoping']],
           labels=[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]],
           names=[u'image-size', u'composite-algorithm'])


Make a grouping structure of the data so we can pull out the actual data values for each trial.

In [10]:
grouped_data = data.groupby(['image-size', 'composite-algorithm'])

Plot data.

In [11]:
image_size = 'HDTV'
algorithms = ['2-3 Swap', 'Naive', 'Remainder']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    # Some adjustments for correct label placement
    if algorithm == 'Remainder':
        vert_shift = '-10%'
    elif algorithm == 'Naive':
        vert_shift = '80%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '&#160;',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'5px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 165.90451812500004 L 61.912708376191546 145.584802725 L 69.13654821749239 140.65389362499999 L 75.731397299034086 149.50370995 L 87.414932308472999 144.26104127500003 L 97.535670681706364 165.30846700000001 L 102.13058970652057 143.73141069999997 L 114.44837905789791 144.03474645000003 L 121.67221889919875 144.0086666 L 131.36108743408954 142.9883524 L 139.95060299017936 142.86741392499997 L 150.07134136341276 163.29153235000001 L 156.86331136637091 141.00862530000001 L 166.9840497396043 142.36320534999999 L 175.91117365596611 142.7107397 L 183.89675811579593 140.58172525000001 L 193.82777686165332 142.60388924999998 L 202.60701204511912 161.54308970000002 L 210.47407740095119 137.00361777499998 L 219.51972042131067 139.99930500000002 L 228.4468443376725 140.35416490000003 L 237.18659374188502 137.88957447500002 L 246.36344754335968 138.33104200000005 L 255.14268272682548 157.3047517 L 263.54163062725905 130.98153622500004 L 272.52762285847336 129.94395067500002 L 281.40242164223406 130.45249102500003 L 290.09655136477687 127.1954032 L 298.89911822506605 125.94524847500001 L 307.67835340853179 140.97819695000001 L 316.341849401498 111.74280775000001 L 325.06329354017976 106.94691399999999 L 333.9380923239404 98.6835375 L 342.63222204648326 87.747171500000022 L 351.43478890677238 85.254087249999969 L 360.21402409023818 90.183000500000006" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap  <path d="M 45.0 166.34327620000002 L 61.912708376191546 123.92908874999998 L 69.13654821749239 122.62002299999999 L 75.731397299034086 122.51432100000002 L 87.414932308472999 121.7687205 L 97.535670681706364 165.878003425 L 102.13058970652057 124.07403875000004 L 114.44837905789791 122.02427850000005 L 121.67221889919875 121.12458500000002 L 131.36108743408954 121.77033724999998 L 139.95060299017936 121.20815425000001 L 150.07134136341276 164.96098839999999 L 156.86331136637091 121.74586300000003 L 166.9840497396043 121.22047500000002 L 175.91117365596611 121.40049175000004 L 183.89675811579593 119.63444324999999 L 193.82777686165332 67.780197500000028 L 202.60701204511912 164.59705240000002 L 210.47407740095119 121.10038950000001 L 219.51972042131067 120.70618125000001 L 228.4468443376725 119.88503949999999 L 237.18659374188502 120.56312675 L 246.36344754335968 119.60277725000002 L 255.14268272682548 164.68583427499999 L 263.54163062725905 118.79317574999999 L 272.52762285847336 116.57900874999999 L 281.40242164223406 119.36896174999998 L 290.09655136477687 117.656712 L 298.89911822506605 116.84917324999999 L 307.67835340853179 162.72604337499999 L 316.341849401498 117.58512899999999 L 325.06329354017976 116.5427155 L 333.9380923239404 111.23381025 L 342.63222204648326 105.11179125000001 L 351.43478890677238 110.22590600000002 L 360.21402409023818 160.00868244999998" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive  <circle r="2.0

In [12]:
toyplot.pdf.render(canvas, 'no-compress-hdtv.pdf')

In [13]:
image_size = '8K UHD'
algorithms = ['2-3 Swap', 'Naive', 'Remainder']

canvas = toyplot.Canvas('4.5in', '3in')

axes = canvas.cartesian(
    xlabel='Number of Processes',
    ylabel='Partial Composite Time (seconds)',
    xscale='log',
    bounds=(45,-15,15,-50),
)

#axes.x.ticks.locator = toyplot.locator.Log(base=2, format='{:.0f}')
axes.x.ticks.locator = toyplot.locator.Explicit(
    locations=[128, 256, 512, 1024, 2048, 4096, 8192],
)
axes.y.domain.min = 0

for algorithm in algorithms:
    # Plot line following average
    data_series = average_partial_composite[image_size, algorithm]
    x = data_series.index
    y = numpy.array(data_series)
    axes.plot(x, y)
    
    if algorithm == 'Remainder':
        vert_shift = '-10%'
    elif algorithm == 'Naive':
        vert_shift = '80%'
    elif algorithm == '2-3 Swap':
        vert_shift = '100%'
    else:
        vert_shift = '0'
    
    # Put label at end of series
    axes.text(
        x[-1],
        y[-1],
        algorithm + '&#160;',
        style={
            'text-anchor':'start',
            '-toyplot-anchor-shift':'5px',
            'baseline-shift': vert_shift,
        },
    )
    
    # Put a dot at every recorded sample.
    scatter_data = data.loc[grouped_data.groups[(image_size, algorithm)]]
    x = scatter_data['num-processes']
    y = scatter_data['partial-composite-seconds']
    axes.scatterplot(x, y, opacity=1)

<path d="M 45.0 125.05329336989151 L 61.912708376191546 92.647731094866586 L 69.13654821749239 88.886970841786194 L 75.731397299034086 98.297656163413222 L 87.414932308472999 92.572994077764889 L 97.535670681706364 123.69733499419341 L 102.13058970652057 81.982420261850493 L 114.44837905789791 90.592919820935506 L 121.67221889919875 88.085549312606304 L 131.36108743408954 87.891533950453677 L 139.95060299017936 87.953483469302228 L 150.07134136341276 123.60696284267286 L 156.86331136637091 78.173465109891424 L 166.9840497396043 89.436457163784226 L 175.91117365596611 85.924032275940476 L 183.89675811579593 84.517891028946195 L 193.82777686165332 85.924677023751542 L 202.60701204511912 122.88516766816208 L 210.47407740095119 78.838092645152585 L 219.51972042131067 88.521398832900474 L 228.4468443376725 84.634805298689756 L 237.18659374188502 84.758919252323821 L 246.36344754335968 83.374000954111125 L 255.14268272682548 122.23590662239846 L 263.54163062725905 75.142559378960399 L 272.52762285847336 77.919756846229049 L 281.40242164223406 81.408755896936768 L 290.09655136477687 80.302637298034455 L 298.89911822506605 79.757395565792677 L 307.67835340853179 119.88252338295032 L 316.341849401498 72.870360634724818 L 325.06329354017976 75.033382082949856 L 333.9380923239404 75.764311184784333 L 342.63222204648326 76.153792591664541 L 351.43478890677238 76.439845703849684 L 360.21402409023818 114.53675809428354" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 2-3 Swap  <path d="M 45.0 124.74064441049903 L 61.912708376191546 55.361750263826181 L 69.13654821749239 54.360994203052201 L 75.731397299034086 54.588482722397003 L 87.414932308472999 53.847076468632373 L 97.535670681706364 124.00305291461667 L 102.13058970652057 55.281317974393168 L 114.44837905789791 53.885976253234517 L 121.67221889919875 53.216513109390277 L 131.36108743408954 52.589657050061859 L 139.95060299017936 47.538810155982695 L 150.07134136341276 123.15542445897563 L 156.86331136637091 52.734993952477559 L 166.9840497396043 52.822357280879672 L 175.91117365596611 51.944264491164788 L 183.89675811579593 52.362867007512399 L 193.82777686165332 51.73590349021552 L 202.60701204511912 123.50358827696208 L 210.47407740095119 52.84734125855929 L 219.51972042131067 51.854429629486845 L 228.4468443376725 51.937225994227106 L 237.18659374188502 48.585504498296558 L 246.36344754335968 50.890692838866023 L 255.14268272682548 123.04119663844489 L 263.54163062725905 52.591806209432193 L 272.52762285847336 41.733017304106042 L 281.40242164223406 49.247230668407795 L 290.09655136477687 49.338086380786713 L 298.89911822506605 49.604904516607775 L 307.67835340853179 121.41605605161838 L 316.341849401498 50.856789849799739 L 325.06329354017976 50.729290970157535 L 333.9380923239404 48.520707343282439 L 342.63222204648326 49.159867340005675 L 351.43478890677238 44.4666405651422 L 360.21402409023818 121.5340449010471" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:1.0;stroke-width:2.0"> Naive  <g class="toyplot-Datum" style=

In [14]:
toyplot.pdf.render(canvas, 'no-compress-8k.pdf')